<a href="https://colab.research.google.com/github/Shawty2084/CSE523-Machine-Learning-2022-Mandelbrot_Set/blob/main/Catboost_end.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import glob
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dir = os.chdir("/content/drive/MyDrive/BBC News Summary")
print(f'Directory change to {os.getcwd()}')

Directory change to /content/drive/MyDrive/BBC News Summary


In [ ]:
def import_data(folders_n, def_path):
  filename = [path + "/News Articles/" + folder + "/" + "*.txt"
     for path,folder in zip([def_path]*len(folders_n), folders_n)]
  temp_doc = []
  print(filename)
  # tags 
  for x in filename:
    data = glob.glob(x) 
    for y in data:
      with open(y, encoding="latin-1") as f:
        topic = x.split('/')[len(x.split('/'))-2] 
        # print('me',topic)
        line = f.readlines()
        # head = line[0].strip()
        body = ''.join([l.strip() for l in line[1:]])
        temp_doc.append([topic,body])
    print(f"Loading data from \033[1m{topic}\033[0m directory")
    print("\nEntire Data is loaded successfully")

  return temp_doc

In [ ]:
folders_n = ['business','entertainment','politics','sport','tech']
doc = import_data(folders_n = folders_n, def_path = os.getcwd())
# docs = load_data(folder_names = folders_n, root_path = os.getcwd())


['/content/drive/MyDrive/BBC News Summary/News Articles/business/*.txt', '/content/drive/MyDrive/BBC News Summary/News Articles/entertainment/*.txt', '/content/drive/MyDrive/BBC News Summary/News Articles/politics/*.txt', '/content/drive/MyDrive/BBC News Summary/News Articles/sport/*.txt', '/content/drive/MyDrive/BBC News Summary/News Articles/tech/*.txt']
Loading data from business directory

Entire Data is loaded successfully
Loading data from entertainment directory

Entire Data is loaded successfully
Loading data from politics directory

Entire Data is loaded successfully
Loading data from sport directory

Entire Data is loaded successfully
Loading data from tech directory

Entire Data is loaded successfully


In [ ]:
# print(type(doc))
doc1 = pd.DataFrame(doc, columns = ['category','text'])
length = len(doc1)
# print(length)
doc1.head(length)
# print(doc1)

,category,text
0,business,Billions of farmers' livelihoods are at risk f...
1,business,"Circuit City Stores, the second-largest electr..."
2,business,Ask Jeeves has become the third leading online...
3,business,Japan's economy teetered on the brink of a tec...
4,business,China's efforts to stop the economy from overh...
...,...,...
2220,tech,"Gambling is hugely popular, especially with te..."
2221,tech,A group of artists in Poland has taken the cac...
2222,tech,The number of personal computers worldwide is ...
2223,tech,The proportion of surfers using Microsoft's In...


In [ ]:
import re
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from spacy.lang.en import English
import spacy

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
spacy_english = English()
from tqdm import tqdm
nltk.download('stopwords')
spacy_english = spacy.load('en_core_web_sm')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
def preprocess_text(text, return_string=False):
    # Tokenize text using Spacy tokenizer
    tokenizer = spacy_english.tokenizer
    tokens = [str(token) for token in tokenizer(text)]
    
    # Lemmatize tokens using Spacy lemmatizer
    lemmatizer = spacy.load('en_core_web_sm')
    doc = lemmatizer(text)
    lemmas = [token.lemma_ for token in doc]
    
    # Remove punctuation
    punctuation_pattern = re.compile('[^A-z ]')
    no_punct_text = [re.sub(punctuation_pattern, '', token) for token in lemmas]
    no_punct_text = [token for token in no_punct_text if token != '']
    
    # Remove stop words
    stopwords_list = stopwords.words('english')
    stopwords_list.extend(['I', 's'])
    no_stopwords_text = [token for token in no_punct_text if token not in stopwords_list]
    
    if return_string:
        return ' '.join(no_stopwords_text)
    else:
        return no_stopwords_text

new_texts = [preprocess_text(text) for text in tqdm(doc1['text'])]


100%|██████████| 2225/2225 [35:01<00:00,  1.06it/s]


In [ ]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 10.8 MB/s eta 0:00:00


In [ ]:

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split  

df = doc1
# Load and preprocess data
train_size = int(len(df) * 0.8)
train_data, test_data = train_test_split(df, test_size=0.1, random_state=42, shuffle=True)

# Remove stop words and perform lemmatization
# code for data preprocessing goes here

# Extract features using TF-IDF vectorization
tfidf = TfidfVectorizer(stop_words='english')
X_train = tfidf.fit_transform(train_data['text'])
y_train = train_data['category']
X_test = tfidf.transform(test_data['text'])
y_test = test_data['category']

# Define and train the CatBoost model
model = CatBoostClassifier(iterations=100,
                           depth=2,
                           loss_function='MultiClass',
                           verbose=False,
                           random_seed=42)
# X_train_dense = X_train.toarray()
# train_pool = Pool(X_train_dense, y_train, text_features=[0])
# # train_pool = Pool(X_train, y_train, text_features=[0])
# test_pool = Pool(X_test, y_test, text_features=[0])
train_pool = Pool(X_train, y_train)
test_pool = Pool(X_test, y_test)

model.fit(train_pool)

# Make predictions on test data and evaluate the model
y_pred = model.predict(test_pool)
accuracy = accuracy_score(y_test, y_pred)
confusion = confusion_matrix(y_test, y_pred)
print('Accuracy:', accuracy)
print('Confusion Matrix:\n', confusion)

# # Extract the training and validation loss values from the logs
# train_loss = model.get_best_score()['validation']['MultiClass']
# val_loss = model.get_best_score()['learn']['MultiClass']

# # Plot the loss function graph
# plt.plot(train_loss, label='Training Loss')
# plt.plot(val_loss, label='Validation Loss')
# plt.title('CatBoost Loss Function Graph')
# plt.xlabel('Iteration')
# plt.ylabel('Loss')
# plt.legend()
# plt.show()


Accuracy: 0.9461883408071748
Confusion Matrix:
 [[56  0  0  1  1]
 [ 1 39  0  3  0]
 [ 2  0 31  3  0]
 [ 0  0  0 49  0]
 [ 0  0  1  0 36]]


KeyError: ignored

In [ ]:
from catboost import CatBoostClassifier, Pool, cv
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, make_scorer
from tqdm import tqdm

df = doc1
# Load and preprocess data
train_size = int(len(df) * 0.8)
train_data, test_data = train_test_split(df, test_size=0.1, random_state=42, shuffle=True)

# Extract features using TF-IDF vectorization
tfidf = TfidfVectorizer(stop_words='english')
X_train = tfidf.fit_transform(train_data['text'])
y_train = train_data['category']
X_test = tfidf.transform(test_data['text'])
y_test = test_data['category']

# Create a pool object for training
train_pool = Pool(X_train, y_train)

# Define a custom scorer for GridSearchCV
scorer = make_scorer(accuracy_score, greater_is_better=True)

# Define the parameter grid to search over
param_grid = {'iterations': [25, 50, 100],
              'depth': [2, 3, 4],
              'learning_rate': [0.1, 0.01, 0.001]}

# Create a CatBoost classifier object
model = CatBoostClassifier(loss_function='MultiClass', verbose=False, random_seed=42)

# Perform grid search to find the best hyperparameters
grid_search = GridSearchCV(model, param_grid, cv=3, scoring=scorer, n_jobs=-1)

# Add a progress bar to the grid search
grid_search.fit(X_train, y_train)

# Print the best hyperparameters and corresponding accuracy score
print('Best Hyperparameters:', grid_search.best_params_)
print('Best Accuracy Score:', grid_search.best_score_)


Best Hyperparameters: {'depth': 4, 'iterations': 100, 'learning_rate': 0.1}
Best Accuracy Score: 0.9250741395769181


In [ ]:
import matplotlib.pyplot as plt

# Enable logging for the training process
model.set_params(logging_level='Verbose')

# Train the model
model.fit(train_pool, eval_set=test_pool, plot=False)

# Retrieve the training results
results = model.get_evals_result()

# Extract the loss values
train_loss = results['learn']['MultiClass']
test_loss = results['validation']['MultiClass']

# Plot the loss function
plt.figure(figsize=(10, 6))
plt.plot(train_loss, label='Training Loss')
plt.plot(test_loss, label='Validation Loss')
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.title('CatBoost Classifier Loss Function')
plt.legend()
plt.grid()
plt.show()


CatBoostError: ignored